In [32]:
!pip install azure.ai.formrecognizer
!pip install azure.ai.documentintelligence 

In [17]:
from azure.core.credentials import AzureKeyCredential
import azure.ai.formrecognizer
from azure.ai.formrecognizer import DocumentAnalysisClient

In [ ]:
endpoint = #
key = #

In [18]:
document_analysis_client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(key))

In [4]:
# Analyze a document at a URL:
url = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"


# Replace with your actual url:
# If you use the URL of a public website, to find more URLs, please visit: https://aka.ms/V3.1-more-URLs 
# If you analyze a document in Blob Storage, you need to generate Public SAS URL, please visit: https://aka.ms/create-sas-tokens

poller = document_analysis_client.begin_analyze_document_from_url("prebuilt-layout", document_url=url)
result = poller.result()
#print(result) ## this worked 

AnalyzeResult(api_version=2023-07-31, model_id=prebuilt-layout, content=UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549
FORM 10-Q :selected: ☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the Quarterly Period Ended March 31, 2020
OR :unselected:
☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the Transition Period From
to
Commission File Number 001-37845
MICROSOFT CORPORATION
WASHINGTON (STATE OF INCORPORATION)
91-1144442
(I.R.S. ID)
ONE MICROSOFT WAY, REDMOND, WASHINGTON 98052-6399 (425) 882-8080 www.microsoft.com/investor
Securities registered pursuant to Section 12(b) of the Act:
Title of each class
Trading Symbol
Name of exchange on which registered
Common stock, $0.00000625 par value per share
MSFT
NASDAQ
2.125% Notes due 2021
MSFT
NASDAQ
3.125% Notes due 2028
MSFT
NASDAQ
2.625% Notes due 2033
MSFT
NASDAQ
Securities registered pursuant to Section 12(g) of the Act:


## Now for Custom Doc SDK code

In [12]:
import azureml.core
import os
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.57.0 to work with azureml-documentclassifier


In [13]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.core.exceptions import HttpResponseError


In [14]:
import os

# Get the current working directory
current_dir = os.getcwd()
print(current_dir)

#Additional Folders need to be added here
base_dir = current_dir + '/CustomClassifier/TRAINING_DOCUMENTS'

if os.path.exists(base_dir):
    print(f"Path exists: {base_dir}")
else:
    print(f"Path does NOT exist: {base_dir}")


for item in os.listdir(base_dir):
    print(item)
    
local_directory =base_dir
print(local_directory)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/comp-instance-docintel/code/Users/jaypadhya
Path exists: /mnt/batch/tasks/shared/LS_root/mounts/clusters/comp-instance-docintel/code/Users/jaypadhya/CustomClassifier/TRAINING_DOCUMENTS
Commercial invoice
Forwarders Cargo Receipt
Packaging List Doc
Sea Waybill
/mnt/batch/tasks/shared/LS_root/mounts/clusters/comp-instance-docintel/code/Users/jaypadhya/CustomClassifier/TRAINING_DOCUMENTS


In [54]:
def analyze_layout():
# [START analyze_layout]

    document_intelligence_client = DocumentIntelligenceClient(endpoint=endpoint, credential=AzureKeyCredential(key))

    # Create arrays to store the incompatible files
    incompatible_files = []

    # Iterate through files in the local directory and analyze each document
    for root, dirs, files in os.walk(local_directory):
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            for file in os.listdir(dir_path):
                document_file_path = os.path.join(dir_path, file)
                if not file.endswith((".json", ".jsonl")):
                    print(f"Analyzing document in {document_file_path}")
                    ocr_json_file_path = document_file_path + ".ocr.json"
                    try:
                        with open(document_file_path, "rb") as f:
                            # Use begin_analyze_document to start the analysis process, and use a callback in order to recieve the raw response
                            poller = document_intelligence_client.begin_analyze_document(
                                "prebuilt-layout", analyze_request=f, content_type="application/octet-stream", cls=lambda raw_response, _, headers: create_ocr_json(ocr_json_file_path, raw_response)
                            )
                    except HttpResponseError as error:
                        print(f"Analysis of {file} failed: {error.error}\n\nSkipping to next file...")
                        incompatible_files.append(document_file_path)
                        continue 
                    result = poller.result()

    # Print the list of incompatible files
    if len(incompatible_files) > 0:
        print("\nThe following files were skipped as they are corrupted or the format is unsupported:")
        for file in incompatible_files:
            print(f"\t{file}")
        print("Please visit the following link for more information on supported file types and sizes. \nhttps://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/concept-custom-classifier?view=doc-intel-4.0.0#input-requirements")
    
    print("Batch layout analysis completed!")
# [END analyze_layout]

def create_ocr_json(ocr_json_file_path, raw_response):
# [START create_ocr_json]
    with open(ocr_json_file_path, "w", encoding="utf-8") as f:
        f.write(raw_response.http_response.body().decode("utf-8"))
        print(f"\tOutput saved to {ocr_json_file_path}")
# [END create_ocr_json]

In [55]:
if __name__ == "__main__":
    analyze_layout()

Analyzing document in /mnt/batch/tasks/shared/LS_root/mounts/clusters/comp-instance-docintel/code/Users/jaypadhya/CustomClassifier/TRAINING_DOCUMENTS/Commercial invoice/.amlignore
Analysis of .amlignore failed: (InvalidRequest) Invalid request.
Code: InvalidRequest
Message: Invalid request.
Inner error: {
    "code": "InvalidContent",
    "message": "The file is corrupted or format is unsupported. Refer to documentation for the list of supported formats."
}

Skipping to next file...
Analyzing document in /mnt/batch/tasks/shared/LS_root/mounts/clusters/comp-instance-docintel/code/Users/jaypadhya/CustomClassifier/TRAINING_DOCUMENTS/Commercial invoice/.amlignore.amltmp
Analysis of .amlignore.amltmp failed: (InvalidRequest) Invalid request.
Code: InvalidRequest
Message: Invalid request.
Inner error: {
    "code": "InvalidContent",
    "message": "The file is corrupted or format is unsupported. Refer to documentation for the list of supported formats."
}

Skipping to next file...
Analyzing 

## Uploading the OCR to Blobs Storage and then use Custom Classification Model 

In [ ]:
from azure.storage.blob import BlobServiceClient, ContainerClient
# Account URL and key
account_url = ""#"https://storageemaildocs.blob.core.windows.net"
account_key = "" # Replace with your storage account key
container_name = ""

# Create a BlobServiceClient
blob_service_client = BlobServiceClient(account_url=account_url, credential=account_key)

# Create a ContainerClient
container_client = blob_service_client.get_container_client(container_name)

# Create the container if it does not already exist
if not container_client.exists():
    print(f"Container {container_name} does not exist. Creating container...")
    container_client.create_container()
    print(f"\tContainer {container_name} created!")


In [15]:
def upload_documents():
# [START upload_documents]
    # Create arrays to store the incompatible files
    incompatible_files = []

    # List all files in the local directory
    for root, dirs, files in os.walk(local_directory):
        for dir in dirs:
            jsonl_data = []
            dir_path = os.path.join(root, dir)
            for file in os.listdir(dir_path):
                local_file_path = os.path.join(dir_path, file)
                ocr_json_file_path = local_file_path + ".ocr.json"
                if ( file.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".heif", ".pdf", ".docx", ".xlsx", ".pptx")) 
                    and os.path.isfile(ocr_json_file_path)):
                        upload_file_to_blob(local_file_path, jsonl_data)
                        upload_file_to_blob(ocr_json_file_path)
                elif not file.endswith((".ocr.json", ".jsonl")):
                    incompatible_files.append(local_file_path)

            # Write the .jsonl file as long as there are at least 5 training files per document type
            if len(jsonl_data) >= 5:
                jsonl_file_path = os.path.join(local_directory, f"{dir}.jsonl")
                with open(jsonl_file_path, "w") as f:
                    for item in jsonl_data:
                        f.write(json.dumps(item) + "\n")
                upload_file_to_blob(jsonl_file_path)

    # Print the list of incompatible files
    if len(incompatible_files) > 0:
        print("\nThe following files are not of a supported file type, missing a corresponding .ocr.json file, or both:")
        for local_file_path in incompatible_files:
            print(f"\t{local_file_path}")
        print("Please ensure you run analyze_layout.py to create .ocr.json files before uploading documents. \nVisit the following link for more information on supported file types and sizes. \nhttps://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/concept-custom-classifier?view=doc-intel-4.0.0#input-requirements")
    
    print("Batch upload completed!")
# [END upload_documents]
    
def upload_file_to_blob(local_file_path, jsonl_data=None):
# [START upload_file_to_blob]
    blob_name = os.path.relpath(local_file_path, local_directory).replace("\\", "/")
    if jsonl_data is not None:
        jsonl_data.append({"file": f"{blob_name}"})
    blob_client = container_client.get_blob_client(blob_name)
    # Upload the file to Azure Blob Storage
    with open(local_file_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded {local_file_path} to {blob_name} in container {container_name}")
# [END upload_file_to_blob]

if __name__ == "__main__":
    upload_documents()

Uploaded /mnt/batch/tasks/shared/LS_root/mounts/clusters/comp-instance-docintel/code/Users/jaypadhya/CustomClassifier/TRAINING_DOCUMENTS/Commercial invoice/Commercial Invoice-4700567821 4700567822-POUCHEN_WWW_PNH1224951 USA - MER-X23005-BUYER DOC.PDF to Commercial invoice/Commercial Invoice-4700567821 4700567822-POUCHEN_WWW_PNH1224951 USA - MER-X23005-BUYER DOC.PDF in container emaildocs
Uploaded /mnt/batch/tasks/shared/LS_root/mounts/clusters/comp-instance-docintel/code/Users/jaypadhya/CustomClassifier/TRAINING_DOCUMENTS/Commercial invoice/Commercial Invoice-4700567821 4700567822-POUCHEN_WWW_PNH1224951 USA - MER-X23005-BUYER DOC.PDF.ocr.json to Commercial invoice/Commercial Invoice-4700567821 4700567822-POUCHEN_WWW_PNH1224951 USA - MER-X23005-BUYER DOC.PDF.ocr.json in container emaildocs
Uploaded /mnt/batch/tasks/shared/LS_root/mounts/clusters/comp-instance-docintel/code/Users/jaypadhya/CustomClassifier/TRAINING_DOCUMENTS/Commercial invoice/Commercial Invoice-4700571518-SHC-20230119-U

## Build the Custom classifier Model here 

In [14]:
import os
from azure.ai.documentintelligence import DocumentIntelligenceAdministrationClient
from azure.ai.documentintelligence.models import (AzureBlobFileListContentSource,ClassifierDocumentTypeDetails,BuildDocumentClassifierRequest)
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient, ContainerSasPermissions, generate_container_sas
from datetime import datetime, timezone, timedelta
import uuid


In [20]:
def build_classifier():
# [START build_classifier]
    base_classifier_id =None # os.environ["BASE_CLASSIFIER_ID"] if os.environ["BASE_CLASSIFIER_ID"] else None
    classifier_description = None #os.environ["CLASSIFIER_DESCRIPTION"] if os.environ["CLASSIFIER_DESCRIPTION"] else None
    document_model_admin_client, container_client = create_clients()
    container_sas_url = create_container_sas_url(container_client)

    poller = document_model_admin_client.begin_build_classifier(
        BuildDocumentClassifierRequest(
            classifier_id=str(uuid.uuid4()),
            base_classifier_id=base_classifier_id,
            description=classifier_description,
            doc_types= get_doctypes(container_client, container_sas_url),
        )
    )
    result = poller.result()
    print_classifier_results(result)
# [END build_classifier]

In [ ]:
from azure.storage.blob import BlobServiceClient, ContainerClient


def create_clients():
# [START create_clients]
    endpoint = "" #DocIntelligence Service
    key = ""##DocIntelligence key
    
    container_name = ""
    # Create a BlobServiceClient
    blob_service_client = BlobServiceClient(account_url=account_url, credential=account_key)
    container_client = blob_service_client.get_container_client(container_name)

    document_model_admin_client = DocumentIntelligenceAdministrationClient(endpoint=endpoint, credential=AzureKeyCredential(key))
    
    
    return document_model_admin_client, container_client


In [22]:
def get_doctypes(container_client, container_sas_url):
# [START get_doctypes]
    doc_types = {}
    doc_types_list = []

    blob_list = container_client.walk_blobs()
    for blob in blob_list:
        if blob.name.endswith(".jsonl"):
            doc_type = os.path.splitext(blob.name)[0]
            doc_types_list.append(doc_type)

    for doc_type in doc_types_list:
        doc_types[doc_type] = ClassifierDocumentTypeDetails(
            azure_blob_file_list_source=AzureBlobFileListContentSource(
                container_url=container_sas_url, 
                file_list=f"{doc_type}.jsonl"
            )
        )
    return doc_types
    

In [23]:
def create_container_sas_url(container_client):
# [START create_container_sas_url]
    # Define the SAS token permissions
    sas_permissions=ContainerSasPermissions(read=True, list=True)

    # Define the expiry time and start time for the SAS token
    start_time = datetime.now(timezone.utc) - timedelta(minutes=1)
    expiry_time = datetime.now(timezone.utc) + timedelta(minutes=5)

    # Generate the container SAS token
    container_sas_token = generate_container_sas(
        container_client.account_name,
        container_client.container_name,
        account_key=container_client.credential.account_key,
        permission=sas_permissions,
        expiry=expiry_time,
        start=start_time,
    )
    # Create the container sas URL by appending the token to the container url
    container_sas_url = f"{container_client.url}?{container_sas_token}"

    return container_sas_url
# [END create_container_sas_url]

In [ ]:
def print_classifier_results(result):
# [START print_classifier_results]
    print(f"Classifier ID: {result.classifier_id}")
    print(f"API version used to build the classifier model: {result.api_version}")
    print(f"Classifier description: {result.description}")
    print(f"Document classes used for training the model:")
    for doc_type in result.doc_types.items():
        print(f"Document type: {doc_type}")
        
# [START print_classifier_results]
        
if __name__ == "__main__":
    build_classifier()

## Classification of Documents

In [36]:
import logging, json, os, time,uuid
from azure.core.exceptions import HttpResponseError
from requests import post, get
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult


In [ ]:
DOCUMENT_INTELLIGENCE_KEY = key
ENDPOINT = endpoint

CLASSIFIER_ID ="" 
API_TYPE = ""##"documentClassifiers"
API_VERSION = "2024-02-29-preview"


In [ ]:
import os

# Get the current working directory
current_dir = os.getcwd()
print(current_dir)

#Additional Folders need to be added here
train_doc_dir = current_dir + '/CustomClassifier/TESTING_DOCUMENTS'

if os.path.exists(train_doc_dir):
    print(f"Path exists: {train_doc_dir}")
else:
    print(f"Path does NOT exist: {train_doc_dir}")


for item in os.listdir(train_doc_dir):
    print(item)
    
local_directory =train_doc_dir
#print(local_directory)
TESTING_DATA = train_doc_dir #This works

In [34]:
def classify_document(classifier_id, doc_path):

    document_intelligence_client = DocumentIntelligenceClient(endpoint=endpoint, credential=AzureKeyCredential(key))
    with open(doc_path, "rb") as f:
        poller = document_intelligence_client.begin_classify_document(
            classifier_id, classify_request=f, content_type="application/pdf"
        )
    result: AnalyzeResult = poller.result()

    print("----Classified documents----")
    if result.documents:
        for doc in result.documents:
            if doc.bounding_regions:
                print(
                    f"Found document of type '{doc.doc_type or 'N/A'}' with a confidence of {doc.confidence} contained on "
                    f"the following pages: {[region.page_number for region in doc.bounding_regions]}"
                )
    # [END classify_document]

In [42]:
for document in os.listdir(TESTING_DATA):
    print(document)
    # Check if the file has a .pdf extension (case-insensitive)
    if document.lower().endswith(".pdf"):
        doc_path = os.path.join(TESTING_DATA, document)
        request = classify_document(CLASSIFIER_ID, doc_path)
    else:
        print(f"Skipping {document}, as it is not a PDF file.")

.amlignore
Skipping .amlignore, as it is not a PDF file.
.amlignore.amltmp
Skipping .amlignore.amltmp, as it is not a PDF file.
Commercial Invoice-4600345298 4600346678...-0014851.pdf
----Classified documents----
Found document of type 'Sea Waybill' with a confidence of 0.914 contained on the following pages: [1, 2]
Commercial Invoice-4600345298 4600346678...-0014852.pdf
----Classified documents----
Found document of type 'Sea Waybill' with a confidence of 0.886 contained on the following pages: [1, 2, 3]
Commercial Invoice-4600345298 4600346678...-0014853.pdf
----Classified documents----
Found document of type 'Sea Waybill' with a confidence of 0.9 contained on the following pages: [1, 2, 3]
Commercial Invoice-4600345298 4600346678...-0014854.pdf
----Classified documents----
Found document of type 'Sea Waybill' with a confidence of 0.854 contained on the following pages: [1, 2, 3]
Commercial Invoice-4600345298 4600346678...-0014855.pdf
----Classified documents----
Found document of ty

In [43]:
if __name__ == "__main__":
    try:
        for document in os.listdir(TESTING_DATA):
            print(document)
            # Check if the file has a .pdf extension (case-insensitive)
            if document.lower().endswith(".pdf"):
                doc_path = os.path.join(TESTING_DATA, document)
                request = classify_document(CLASSIFIER_ID, doc_path)
            else:
                print(f"Skipping {document}, as it is not a PDF file.")
    except HttpResponseError as error:
        # Examples of how to check an HttpResponseError
        # Check by error code:
        if error.error is not None:
            if error.error.code == "InvalidImage":
                print(f"Received an invalid image error: {error.error}")
            if error.error.code == "InvalidRequest":
                print(f"Received an invalid request error: {error.error}")
            # Raise the error again after printing it
            raise
        # If the inner error is None and then it is possible to check the message to get more information:
        if "Invalid request".casefold() in error.message.casefold():
            print(f"Uh-oh! Seems there was an invalid request: {error}")
        # Raise the error again
        raise

.amlignore
Skipping .amlignore, as it is not a PDF file.
.amlignore.amltmp
Skipping .amlignore.amltmp, as it is not a PDF file.
Commercial Invoice-4600345298 4600346678...-0014851.pdf
----Classified documents----
Found document of type 'Sea Waybill' with a confidence of 0.914 contained on the following pages: [1, 2]
Commercial Invoice-4600345298 4600346678...-0014852.pdf
----Classified documents----
Found document of type 'Sea Waybill' with a confidence of 0.886 contained on the following pages: [1, 2, 3]
Commercial Invoice-4600345298 4600346678...-0014853.pdf
----Classified documents----
Found document of type 'Sea Waybill' with a confidence of 0.9 contained on the following pages: [1, 2, 3]
Commercial Invoice-4600345298 4600346678...-0014854.pdf
----Classified documents----
Found document of type 'Sea Waybill' with a confidence of 0.854 contained on the following pages: [1, 2, 3]
Commercial Invoice-4600345298 4600346678...-0014855.pdf
----Classified documents----
Found document of ty